# Exceptions

Exceptions contain a payload `args` that must be a single string containing the error message. It can be retrieve by using the readonly named attribute `args` or transforming the exception into `str`.

In [2]:
def access_third(sequence):
    return sequence[3]

sequence = [1, 2]
access_third(sequence)

IndexError: list index out of range

In [6]:
try:
    access_third(sequence)
except LookupError as e:
    print(e)
    print(e.args)
    print(e.args[0])

list index out of range
('list index out of range',)
list index out of range


### Named attributes
Some exceptions can contain named attributes that contain further information regarding the cause of the exception.

In [8]:
try:
    b'\x81'.decode('utf-8')
except UnicodeError as e:
    print(e)
    print("encoding:", e.encoding)
    print("reason:", e.reason)
    print("object:", e.object)
    print("start:", e.start)
    print("end:", e.end)

'utf-8' codec can't decode byte 0x81 in position 0: invalid start byte
encoding: utf-8
reason: invalid start byte
object: b'\x81'
start: 0
end: 1


In [1]:
class MySimpleException(Exception):
    pass

class MyComplexException(Exception):
    def __init__(self, text, other_information):
        super().__init__(text)
        self._info = other_information
    
    @property
    def info(self):
        return self._info
    
    def __str__(self):
        return '{} because {}'.format(self.args[0], self._info)
    
    def __repr__(self):
        return 'MyComplexException(test={!r}, other_information={!r})'.format(self.args[0], self._info)

In [2]:
try:
    raise MySimpleException("I'm a humble simple exception")
except MySimpleException as e:
    print(e)

I'm a humble simple exception


In [3]:
try:
    raise MyComplexException("I'm a more complex exception", 'With even more data to be provided, and accessible')
except MyComplexException as e:
    print(e)
    print(e.args)
    print(e.info)

I'm a more complex exception because With even more data to be provided, and accessible
("I'm a more complex exception",)
With even more data to be provided, and accessible


## Chaining Exceptions
Chaining exceptions can happen in two way `implicit` and `explicit`.

### Implicit chaining
This way of chaining exceptions happens when handling an exception another exception occurs. The way Python associates this two exceptions is by assigning the original exception to the `__context__` attribute of the new exception.

### Explicit chaining
This way of chaining exceptions happens when the handling of an exception occurs inside the handling of another exception. This is also used when translating an exception from one type into another. The way Python associates this two exceptions is by assigning the original exception to the `__cause__` attribute of the new exception.

In [4]:
try:
    raise MySimpleException("I'm a humble simple exception")
except MySimpleException as e:
    raise IOException("WOLOLO")

NameError: name 'IOException' is not defined

In [7]:
try:
    raise MySimpleException("I'm a humble simple exception")
except MySimpleException as e:
    try:
        raise Exception("WOLOLO")
    except Exception as f:
        print(e)
        print(f)
        print(f.__context__ is e)

I'm a humble simple exception
WOLOLO
True


In [11]:
try:
    raise MySimpleException("I'm a humble simple exception")
except MySimpleException as e:
    raise Exception("WOLOLO") from e

Exception: WOLOLO

In [12]:
try:
    raise MySimpleException("I'm a humble simple exception")
except MySimpleException as e:
    try:
        raise Exception("WOLOLO") from e
    except Exception as f:
        print(e)
        print(f)
        print(f.__cause__ is e)

I'm a humble simple exception
WOLOLO
True
